In [16]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [12]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [36]:
code="""
package com.orbitradar.domain.usecase

import com.orbitradar.domain.TleRepositoryInterface
import com.orbitradar.domain.config.TleType
import com.orbitradar.domain.converter.toSatelliteDomain
import com.orbitradar.domain.model.SatelliteDomain
import javax.inject.Inject

class GetSatellites @Inject constructor(
    private val tleRepository: TleRepositoryInterface
) {

    suspend operator fun invoke(vararg tleTypes: TleType): List<SatelliteDomain> =
        tleRepository.getTles(*tleTypes).map { it.toSatelliteDomain() }

    suspend operator fun invoke(searchName: String): List<SatelliteDomain> =
        tleRepository.getTles(searchName).map { it.toSatelliteDomain() }
}
"""

In [39]:
prompt_template=f'''
You will be provided with code written in kotlin programming language which will be a code which is a part of the android project.
Your task is to write unit tests for this code. Use the following specifications marked with the * characters.

* Use mockito kotlin library
* If class name is SampleClassName then unit test class should be named SampleClassNameTest
* Assume using junit4 library
* For mock instances use following naming convention: mockSampleProperty
* Avoid using annotation for generating mock properties
* Don't use @Before and @After annotations and setup methods as junit4 recreates all properties
* Don't use lateinit when creating mock properties
* Create instance of class with name systemUnderTest
* Tests should be granular, avoid writing big tests
* Test functions should be named with given action for example fun `returns list data when no error occurs`(). Avoid camel case naming convention for test functions
* Use Given, When, Then convention
* If there is a need to use mockito kotlin whenever and same whenever statement repeats, then create private helper function with name e.g. assumeNoInternetConnection
* All tests should be completed. If you don't know how to write given unit test then don't write it
* As an output print only generated code, avoid any response messages

Here is a code:
```{code}```
'''

In [40]:
response = get_completion(prompt_template)
print(response)

```
package com.orbitradar.domain.usecase

import com.nhaarman.mockitokotlin2.mock
import com.nhaarman.mockitokotlin2.whenever
import com.orbitradar.domain.TleRepositoryInterface
import com.orbitradar.domain.config.TleType
import com.orbitradar.domain.model.SatelliteDomain
import kotlinx.coroutines.runBlocking
import org.junit.Test

class GetSatellitesTest {

    private val mockTleRepository: TleRepositoryInterface = mock()
    private val systemUnderTest = GetSatellites(mockTleRepository)

    @Test
    fun `returns list of SatelliteDomain when invoking with TleType`() {
        // Given
        val tleTypes = arrayOf(TleType.TYPE1, TleType.TYPE2)
        val mockTles = listOf(mock<Tle>(), mock<Tle>())
        val expectedSatellites = listOf(mock<SatelliteDomain>(), mock<SatelliteDomain>())
        whenever(mockTleRepository.getTles(*tleTypes)).thenReturn(mockTles)
        mockTles.forEachIndexed { index, tle ->
            whenever(tle.toSatelliteDomain()).thenReturn(expectedSatelli